In [ ]:
%load_ext autoreload
%autoreload 2

# from app.config import settings
# from celery import Celery
# from app.service import demo_process
from PIL import Image
import io 
import numpy as np
import requests
import random 
from celery import Celery
from src.core.config import settings
from src.core.config.main import load_ml_settings
from src.api.route import style_transfer_celery_app,style_transfer_task_lib
#from src.workers.style_transfer_workers import celery_app, style_transfer_task_lib,style_transfer_task_info

In [9]:
import os 



In [10]:
os.environ["MODEL_KEY"] = "candy_v1"

In [ ]:
!celery -A src.workers.worker_factory.celery_app worker

In [ ]:
url = "http://127.0.0.1:8000/api/v1/style_transfer"

# Query parameters
params = {
    "model_name": "rain-princess",
    "model_mode": 2
}

# File to upload
files = {
    'file': ('amber.jpg', open('data/amber.jpg', 'rb'), 'image/jpeg')
}

response = requests.post(url, params=params, files=files)

response_json = response.json()

# Print response
print(response_json)

In [42]:
result_id = response_json['result_id']

response = requests.get(f'http://127.0.0.1:8000/result/{result_id}')
with open("image.png", "wb") as f:
    f.write(response.content)

# response = requests.get(f'http://127.0.0.1:8000/result/{result_id}')

# # Create a BytesIO object - this acts like a file in memory
# image_bytes = io.BytesIO(response.content)

# # Now open the image using PIL - this will work because BytesIO is file-like
# pil_image = Image.open(image_bytes)

# # Convert to numpy array
# image_array = np.array(pil_image)

In [20]:
task_0 = style_transfer_task_lib[0]
task_1  = style_transfer_task_lib[1]
task_2 = style_transfer_task_lib[2]

In [27]:
celery_app = Celery('model_tasks', broker=settings.RABBITMQ_URL, backend=settings.REDIS_URL)
model_name = "candy"

@celery_app.task(name=f"task_{model_name}", queue=f"task_{model_name}_queue")
def dynamic_task(image_bytes: bytes) -> str:

    res = image_process(image_bytes)
    return res

In [31]:
@celery_app.task(name=f"task_{model_name}", queue=f"task_{model_name}_queue")
def dynamic_task(image_bytes: bytes) -> str:
    return f"task_{model_name}"

In [ ]:
from celery.apps.worker import Worker
from src.workers.style_transfer_workers import create_worker_task
from src.core.config import settings
from src.ml.style_transfer_service import ImageService
import socket

def start_style_transfer_worker(model_name: str, model_number: int, model_path: str):
    # First, we initialize our image service
    style_transfer_service = ImageService(model_path=model_path)
    
    # Create our worker task and get the Celery app instance
    worker_task, celery_app = create_worker_task(
        settings,
        model_name,
        model_number,
        style_transfer_service
    )
    
    # Configure worker options - these determine how our worker will behave
    worker_options = {
        'queues': [f"task_{model_name}_queue"],
        'loglevel': 'INFO',
        'concurrency': 1,
        'hostname': f"{model_name}-worker@{socket.gethostname()}"
    }
    
    # Create and start the worker using the application's worker method
    worker = celery_app.Worker(**worker_options)
    
    try:
        worker.start()
    except (KeyboardInterrupt, SystemExit):
        print("Shutting down worker gracefully...")
        worker.stop()

if __name__ == "__main__":
    model_type = "style_transfer"
    model_path = "models/candy-8.onnx"
    
    start_style_transfer_worker("candy", 0, model_path)

In [ ]:
for i in range(500):
    i = random.randint(0, 2)
    
    # Query parameters
    params = {
        "model_name": "53",
        "model_mode": 0
    }

    # File to upload
    files = {
        'file': ('1.JPG', open('1.JPG', 'rb'), 'image/jpeg')
    }

    response = requests.post(url, params=params, files=files)

    # Print response
    print(response.json())